In [ ]:
# Config Settings

lang = "auto" #@param ["jp", "en", "auto"]
lang = "id" if (lang == "auto") else lang
ay = 2022 #@param ["2022", "2021"] {type:"raw"}

USE_BORDER = False #@param {type: "boolean"}
BORDER_STYLE = "1px solid red"  #@param {type:"string"}

# Default registered subject list (code, class)
default_list = [(4102,"")]

In [ ]:
#@title
# #@title Updating Library (ipywidgets)

if USE_BORDER:
    !pip install -q -U ipywidgets==8.0.0rc0  # !! restart runtime is required
    #widgetsnbextension 4.0.0rc0, jupyterlab_widgets 3.0.0rc0
    print("WARNING: for now, courses cannot be added interactively while USE_BORDER is set.")
    # print("Restart runtime is required.\nPlease restart runtime after running this cell once.")

from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
#@title
# lang = "id"

# Default registered subject list (code, class)
default_list = [
        (2001,"B(Fall)"),  # Q3 Intro IS
        (2003,"B(autumn)"),  # Q3; Intro BS
        (2005,"B(Fall)"),  # Q3; Intro MS
        (4102,""),  # Q3;  NLP
        (4029,"2"), # Q3;  Special Lectures A, C - 2. Geoparsing
] if lang != "jp" else [(4102,"")]


In [ ]:
#@title No Show Version
#@markdown (here displays nothing when USE_BORDER is set)
from IPython.display import display, Javascript

def set_iframe_maxHeight(h=5000):
  display(Javascript(f"google.colab.output.setIframeHeight(0, true, {{maxHeight: {h}}})"))

# -------------------- #

from ipywidgets import AppLayout

le_screen = AppLayout()
le_screen.pane_heights = [0, "50px", 50]
set_iframe_maxHeight()
if not USE_BORDER:
    display(le_screen)

### Processing Part

In [ ]:
#@title
try:
    if lang is None:
        import locale
        current_locale = locale.getlocale()[0][:2].lower()
    else:
        current_locale = lang
except:
    current_locale = "jp"

# ==========

label_regenerate = {
    "en": "Re-Generate",
    "jp": "再生",
}.get(current_locale, "🔄")

_data_url = {
    "2022jp": "https://raw.githubusercontent.com/lintaoren/naist_course_planner/main/NAIST%20Courses%202022%20by%20Month%20-%20RWS.jp.json",
    "2022en": "https://raw.githubusercontent.com/lintaoren/naist_course_planner/main/NAIST%20Courses%202022%20by%20Month%20-%20RWS.en.json",
    "2021en": "https://raw.githubusercontent.com/lintaoren/naist_course_planner/main/NAIST%20Courses%202021%20-%20RWS.json",
}
data_url = _data_url.get(f"{ay}{lang}", _data_url["2022en"])

_json_path = "_data.json"
!wget -q -O {_json_path} {data_url}

In [ ]:
#@title
from unicodedata import category, normalize
from functools import partial
from itertools import chain
from datetime import date, datetime, timedelta

import pandas as pd
import json
import re
import os


# -------------------- #

from ipywidgets import Button, Layout, jslink, IntText, IntSlider, Label
from ipywidgets import Dropdown, Text, Output, HBox, VBox, GridspecLayout

def expanded():
    return Layout(height='auto', width='auto')

def create_expanded_button(description, button_style, *args, **kwargs):
    return Button(description=description, button_style=button_style, layout=expanded(), *args, **kwargs)

def AxesButton(desc):
    return Button(
        description=desc, layout=Layout(height='auto', width='auto'), disabled=True)


# -------------------- #

def iso_following_week(time_iso):
    return (date.fromisoformat(
                datetime.strptime("/".join(map(str, time_iso[:3])),"%Y/%W/%w")
                        .isoformat().split("T",1)[0]
            ) + timedelta(days=7)
    ).isocalendar()

def iso_same_week(a, b):
    return (a[0] == b[0])  and  (a[1] == b[1])


# -------------------- #

with open(_json_path, "r", encoding="utf8") as i:
    _data = json.load(i)

In [ ]:
#@title
##### Parsing - Subject's Meta  #1
if (len(_data["data"]) == 1):
    df = pd.DataFrame(_data["data"][0]["userData"])
    df.subject_code = df.subject_code.apply(lambda t: int(t.strip("(").rstrip(")")))
    df.subject_ay = df.subject_ay.apply(lambda t: int(t.rstrip("th")))
    df = df.sort_values(["subject_ay", "subject_code"],ascending=[False, True]).reset_index(drop=True)

    assert all(df.subject_title.apply(lambda t: len(t.split(" - "))).le(2)), \
            "Found subject title more than 1 hyphen .."
    df.loc[:,"title_main"], df.loc[:,"title_sub"] = \
        zip(*df.subject_title.apply(lambda t: [*t.split(" - ", 1), ""][:2]).tolist())

    group = df.groupby("subject_code")
    assert all(group.title_main.unique().apply(len).eq(1)), \
            "Found different subject title (main) with same subject code .."
    assert all(group.title_sub.unique().apply(len) == group.apply(len)), \
            "Unique-ness is miscounted."

    df.loc[df.subject_ay.eq(2021)&df.subject_title.eq("Research Presentation - C"), "title_sub"] = "C1"


    ##### Parsing - Time Table
    def todate(t, y):
        if t == "-": return (y, 0, 0, 0)
        m, d, s = map(int, re.match(r"(\d{1,2})/(\d{1,2}) \[(\d)]", t).groups())
        return (*date(y+(m<4), m, d).isocalendar(), s)

    assert df.time_table.apply(lambda i: all(map(lambda t: normalize("NFKC", t) in [t, "-"], i))).all()
    df.time_table = df.time_table.apply(lambda i: list(map(partial(normalize, "NFKC"), i)))
    df.loc[:, "time_iso"] = df.apply(lambda r: list(map(lambda t: todate(t, r.subject_ay), r.time_table)), axis=1)
    df = df[["subject_ay", "subject_code", "title_main", "title_sub", "time_iso"]]



##### Parsing - Subject's Meta  #2
elif (len(_data["data"]) == 12):
    _flat = [
        dict([*zip(["subject_ay", "month"], i["url"].rsplit("/", 2)[-2:]), *d.items()])
        for i in _data["data"] for d in i["userData"]
    ]
    df = pd.DataFrame(_flat)[["subject_ay", "month", "day", "time", "content", "note", "wd"]]
    print(df.loc[~df.time.str[0].isin(list(map(str, range(1,7))))])
    df = df.loc[df.time.str[0].isin(list(map(str, range(1,7))))]

    header = df .content.apply(lambda t: t.split("\n", 1)[0].split("[", 1)[0].strip().rstrip())
    df["subject_title"], df["subject_code"] = zip(*header.apply(lambda t: re.match(r"(.*)\((\d{4})\)", t).groups()).values)
    df.subject_code, df.subject_ay = map(lambda s: s.astype(int), [df.subject_code, df.subject_ay])

    df["title_main"] = df.groupby("subject_code").apply(lambda df: os.path.commonprefix(list(df.subject_title.unique())).strip().rstrip()).loc[df.subject_code.values].values
    df["title_sub"] = df.apply(lambda r: r.subject_title[len(r.title_main):].strip(), axis=1)


    ##### Parsing - Time Table
    def todate(r):
        y, m, d, s = map(int, [r.subject_ay, r.month, r.day, r.time[0]])  # year, month, day, slot
        return (*date(y+(m<4), m, d).isocalendar(), s)

    assert all(t in {'1限目', '2限目', '3限目', '4限目', '5限目', '6限目',
                     '1st', '2nd', '3rd', '5th', '4th', '6th'} for t in df.time.unique())
    df = (df.groupby(["subject_ay", "subject_code", "title_main", "title_sub"])
            .apply(lambda sub: sub.apply(todate, axis=1).tolist())
            .rename("time_iso").reset_index())


##### for tooltip
code2title = dict(df.apply(lambda r: (f"{r.subject_code}{r.title_sub[:1]}", r.title_main), axis=1).tolist())

In [ ]:
#@title
def get_by_week_num(ay, sel):
    if len(sel) <= 0: return list(), set()

    to_show = sorted(chain(*sel.apply(
                lambda r: [(*t, f"{r.subject_code}{r.title_sub[:1]}") for t in r.time_iso], axis=1)))
    by_week_num = []
    prev = None
    row = []
    nosession = set()
    for cur in to_show:
        if cur[1:4] == (0,0,0):
            assert not by_week_num and not row
            nosession.add(cur[-1])
            continue
        if (prev is None) or iso_same_week(prev, cur):
            row.append(cur)
        else:
            by_week_num.append(row)
            # addtional row if weeknum is not consecutive
            if not iso_same_week(iso_following_week(prev), cur):
                by_week_num.append([])
            row = [cur]
        prev = cur
    by_week_num.append(row)
    return by_week_num, nosession

In [ ]:
#@title
def btn_x_click(btn, idx, container, states):
    if states.get("global_lock", False): return;
    for i in range(idx, states["max_idx"]):
        container.children[i].children[1].value = container.children[i+1].children[1].value
    last_one = container.children[states["max_idx"]]
    last_one.children[1].value, last_one.layout.display = None, "none"
    states["max_idx"] -= 1
    chaocheebye()
    

def generate_entry(idx, container, states):
    btn = Button(description="x", layout=expanded())
    btn.on_click(partial(btn_x_click, idx=idx, container=container, states=states))
    dd = Dropdown(layout=expanded())
    hb = HBox([btn, dd], layout=Layout(height='auto', width='auto', align_items="center", display="none"))
    return hb
    

def btn_add_click(btn, container, states):
    if states.get("global_lock", False): return;
    if len(container.children) - 1  <=  states["max_idx"]:  # container.children[0] is btn_add
        container.children = [*container.children, *(
            generate_entry(len(container.children)+i, container, states)
                for i in range(max(len(container.children)-1, 4))
            )]
    states["max_idx"] += 1

    hb = container.children[states["max_idx"]]
    dd = hb.children[1]
    if len(dd.options) <= 0:
        dd.options, dd.value = states.get("dd_list", []), None
    hb.layout.display = None  # deleting display='none' from style

    chaocheebye()


def chaocheebye():
    # 超级白 / Super White
    # reference: https://www.youtube.com/watch?v=OjIm0hmodsE
    le_screen.pane_heights = [0, f"{50+32*states['max_idx']}px", 50]
    

In [ ]:
#@title
states = {
    "max_idx": 0,  # equals to size of subject entries
    "dd_list": df.apply(
        lambda r: (f"({r.subject_code}) {r.title_main} {r.title_sub}",
                   (r.subject_code, r.title_sub)), axis=1).tolist(),
    "global_lock": False,
}

vb = VBox()
le_screen.center = vb

btn_add = Button(description="Add Course")
btn_add.on_click(partial(btn_add_click, container=vb, states=states))
label_now_loading = Label("　　　　[[ Now Loading (re-generating / 再生中) ... ]]")
label_now_loading.layout.display = "none"
vb.children = [HBox([btn_add, label_now_loading])]


In [ ]:
#@title
def now_loading(container, val):
    container.children[0].children[1].layout.display = None if val else "none"

def generate_grid(container, *args, **kwargs):
    now_loading(container, True)
    lis = list(filter(None, map(lambda hb: hb.children[1].value, filter(lambda hb: hb.layout.display is None, container.children[1:]))))

    def f_cond(r):
        if (r.subject_ay != ay): return False
        ret = False
        for t in lis:
            if ((len(t) > 0) and (r.subject_code==t[0]) and
                ((len(t) == 1) or (r.title_sub.startswith(t[1])))
            ):
                ret = True
                break
        return ret

    sel = df.loc[df.apply(f_cond, axis=1)].copy()
    by_week_num, _ = get_by_week_num(ay, sel)
    # print(sel)

    header_count = 4
    grid = GridspecLayout( header_count + sum(map(lambda r: 1 + (len(r)>0), by_week_num)),  16 )

    for i in range(5):
        x, desc  =  i*3+1, datetime.strptime(f"2202{i+1}","%y%W%w").strftime("%A")
        grid[0, x:x+3] = AxesButton(desc)
    grid[1:3, 0] = AxesButton("date + 0")
    for i in range(6):
        grid[1+i//3, 1+i%3] = AxesButton(f"{i+1}限")
    for i in range(1, 5):
        x = i*3+1
        grid[1:3, x:x+3] = AxesButton(f"date + {i+1}")

    i = header_count
    delim = " ;; "
    border_style = BORDER_STYLE or "1px solid #d01020"
    for l in by_week_num:
        for c in l:
            dow, term = c[2], c[3]
            y = i + (term > 3)
            x = ( (dow-1) * 3  + (term-1) % 3 )  + 1
            try:
                pd = grid[y,x].description
                desc = f"{pd}{delim}{c[-1]}";
                style = "danger"
            except KeyError:
                desc, style = f"{c[-1]}", "info" if c[2]%2 else "success"
            grid[y, x] = create_expanded_button(
                desc, style, disabled=True,
                tooltip = delim.join((map(code2title.get, desc.split(delim))))
            )
            ly = grid[y,x].layout
            if term > 3: ly.border_bottom = border_style
            else: ly.border_top = border_style
            if (term%3 == 1): ly.border_left = border_style
            elif (term%3 == 0): ly.border_right = border_style
            
        if len(l) > 0:
            desc = (datetime.strptime(f"{l[0][0]}/{l[0][1]}/0","%Y/%W/%w")
                    - timedelta(days=7)).strftime("%m/%d (日)")
            grid[i:i+2, 0] = AxesButton(desc)
            i += 1
        i += 1
    
    now_loading(container, False)
    return grid

In [ ]:
#@title
# set_iframe_maxHeight()
# grid
def do_regenerate(*args, **kwargs):
    grid = generate_grid(vb)
    btn_gen = grid[0, 0] = AxesButton(label_regenerate)
    btn_gen.disabled = False
    btn_gen.button_style = "warning"
    btn_gen.on_click(do_regenerate)
    le_screen.footer = grid

In [ ]:
#@title
do_regenerate()
now_loading(vb, True)
for t in default_list:
    btn_add.click()
    dropdown = vb.children[states["max_idx"]].children[1]
    try:
        dropdown.value = t
    except TypeError:
        dropdown.value = next(filter(lambda v: (v[0]==t[0]) and (v[1].startswith(t[1])), dropdown._options_values))
do_regenerate()

### Displaying

In [ ]:
#@title
if USE_BORDER:
    set_iframe_maxHeight()
    display(le_screen.footer)